In [ ]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_tavily import TavilySearch

memory = InMemorySaver()

from dotenv import load_dotenv
load_dotenv()

# 构建state schema
class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

# 初始化工具
tool = TavilySearch(max_results=2)
tools = [tool]

# 初始化模型 绑定工具
llm = init_chat_model("google_genai:gemini-2.0-flash")
llm_with_tools = llm.bind_tools(tools)

# 定义Node
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}
# 增加Node
graph_builder.add_node("chatbot", chatbot)

# 定义工具Node
tool_node = ToolNode(tools)
# 增加工具Node
graph_builder.add_node("my_tools", tool_node)

# 增加条件边
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,    # langgraph自带的工具条件函数返回字符串是写死的，因此下面的映射字典，只能写tools和__end__
    {"tools": "my_tools", "__end__": END} # 如果不需要工具调用 直接结束
)
# 增加普通边
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("my_tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
# graph = graph_builder.compile()
# 使用提供的检查点编译 graph，该检查点将在 graph 遍历每个 Node 时将 State 作为检查点
graph = graph_builder.compile(checkpointer=memory)

In [3]:
config = {"configurable": {"thread_id": "1"}}

user_input = "Hi there! My name is Will."

# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Hi there! My name is Will.
================================== Ai Message ==================================

Hi Will! It's nice to meet you. How can I help you today?


In [7]:
user_input = "What is my name?"

# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is my name?
================================== Ai Message ==================================

Your name is Will.


In [8]:
# The only difference is we change the `thread_id` here to "2" instead of "1"
events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    {"configurable": {"thread_id": "2"}},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is my name?
================================== Ai Message ==================================

I am designed to respect user privacy and do not retain any personal information, including your name, between conversations. Therefore, I do not know your name. You would have to tell me again.


In [9]:
snapshot = graph.get_state(config)
snapshot

StateSnapshot(values={'messages': [HumanMessage(content='Hi there! My name is Will.', additional_kwargs={}, response_metadata={}, id='8d4ada4c-7e09-48c6-88b8-e12a4a8a7966'), AIMessage(content="Hi Will! It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--7f790b38-511f-4f28-bc82-6ce576ce91c8-0', usage_metadata={'input_tokens': 8, 'output_tokens': 19, 'total_tokens': 27, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='Remember my name?', additional_kwargs={}, response_metadata={}, id='e24cc393-b192-45b4-8461-9130ac1dac7b'), AIMessage(content='Yes, your name is Will.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--ee426be1-66cc-4315-8fde

In [10]:
#上面的快照包含了当前状态值、相应的配置以及要处理的 next 节点。在我们的情况下，图已经达到 END 状态，所以 next 是空的
snapshot.next  

()

In [ ]:
# 恭喜！您的聊天机器人现在可以借助 LangGraph 的检查点系统在会话间保持对话状态。这为更自然、更具上下文的交互打开了
# 令人兴奋的可能性。LangGraph 的检查点甚至可以处理任意复杂的图状态，这比简单的聊天记忆更具表现力和强大的功能。